#### CUSTOM_AGENT_EXECUTOR:

Goal: Real-World Custom Agent
We’ll build a custom agent using Groq LLM + LangChain, that can:

🔍 Search Wikipedia for any topic

📅 Convert natural dates to a clean format

📊 Summarize text

In [ ]:
from langchain_core.tools import tool

from langchain.agents import initialize_agent, AgentType # This is for default agent initialization
from langchain.agents import AgentExecutor, Tool, create_tool_calling_agent        # This is for Custom Agent execution

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

import requests
from datetime import datetime

#### Tools:

In [2]:
# Wikipedia Search Tool
@tool
def search_wikipedia(query: str) -> str:
    """Searches Wikipedia and returns the first paragraph."""
    response = requests.get(
        f"https://en.wikipedia.org/api/rest_v1/page/summary/{query.replace(' ', '%20')}"
    )
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No summary found.")
    return "Error fetching data."

# Natural Date Converter
@tool
def convert_date(date_str: str) -> str:
    """Converts a natural language date to YYYY-MM-DD format."""
    try:
        dt = datetime.strptime(date_str, "%B %d, %Y")
        return dt.strftime("%Y-%m-%d")
    except Exception:
        return "Please provide a date like 'June 24, 2025'"

# Text Summarizer
@tool
def summarize_text(text: str) -> str:
    """Summarizes the given text in one line."""
    return f"Summary: {text[:150]}..." if len(text) > 150 else text

In [13]:
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)

# Prompt Template:
prompt = PromptTemplate.from_template("""
You are a helpful AI assistant that uses tools when necessary.

Answer the question as best you can. Use tools if needed.

Question: {input}

{agent_scratchpad}
""")

In [19]:
# Chain & Executor

tools = [search_wikipedia, convert_date, summarize_text]

agent = create_tool_calling_agent(llm, tools, prompt)

# This by default sets max iter rate=5, sometimes agent if agent couldn't fetech within 5, it gets stopped.
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)  

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10
)


In [22]:
# Testing results:

agent_executor.invoke({"input": "Explain Artificial Intelligence?"})



> Entering new AgentExecutor chain...

Invoking: `summarize_text` with `{'text': 'Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems can analyze data, learn from experience, and adapt to new situations, enabling them to make decisions and take actions with minimal human intervention.'}`


Summary: Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as v...
Invoking: `summarize_text` with `{'text': 'Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems can analyze data, learn from experience, and adapt to new situations,

{'input': 'Explain Artificial Intelligence?',
 'output': 'Agent stopped due to max iterations.'}